[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/grsart/BiomolComp/blob/main/Pratica5/MSA0.ipynb)

#Alinhamento Múltiplo de Sequências (MSA)

O alinhamento múltiplo de sequências é uma importante ferramenta para a caracterização de regiões conservadas em proteínas de uma família. Este tipo de alinhamento permite a comparação simultânea de sequências fazendo com que a predição de conservação de resíduos seja muito mais confiável. Além disso, o alinhamento múltiplo permite realização de analises filogenéticas de famílias de proteínas (assunto a ser abordado na próxima aula) que fornece informações sobre sua evolução.






##1. Alinhamento Progressivo com ClustalW

A primeira atividade irá utilizar sequências de proteínas obtidas a partir do Uniprot.

Realizaremos o alinhamento múltiplo de diversos membros selecionados desta família.

Neste primeiro momento, utilizaremos o programa **ClustalW**.

A biblioteca Biopython não tem o clustalw integrado, mas contém ferramentas que permitem executar o ClustalW, caso o programa esteja disponível na máquina local.

### Preparando o Ambiente para a Execução

O código ao lado realiza a instalação do biopython no ambiente do Google Colab, além da importação de algumas bibliotecas que serão necessárias.

Na sequência o código faz o download do clustalw a partir [deste link](http://www.clustal.org/download/current/clustalw-2.1-linux-x86_64-libcppstatic.tar.gz), permitindo que seja executado pelo biopython.

In [2]:
#@title Preparando o Ambiente para a Execução

# Installing Biopython and importing required libraries...

!pip3 install biopython
from Bio import SeqIO
from Bio.Align.Applications import ClustalwCommandline
from google.colab import files
import os

# Gettting ClustalW2....

!wget http://www.clustal.org/download/current/clustalw-2.1-linux-x86_64-libcppstatic.tar.gz
!tar -xzf clustalw-2.1-linux-x86_64-libcppstatic.tar.gz
!mv clustalw-2.1-linux-x86_64-libcppstatic/clustalw2 .


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


--2025-09-25 22:00:37--  http://www.clustal.org/download/current/clustalw-2.1-linux-x86_64-libcppstatic.tar.gz
Resolving www.clustal.org (www.clustal.org)... 137.43.93.206
Connecting to www.clustal.org (www.clustal.org)|137.43.93.206|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2468744 (2.4M) [application/x-gzip]
Saving to: ‘clustalw-2.1-linux-x86_64-libcppstatic.tar.gz’

clustalw-2.1-linux- 100%[===================>]   2.35M  2.98MB/s    in 0.8s    

2025-09-25 22:00:38 (2.98 MB/s) - ‘clustalw-2.1-linux-x86_64-libcppstatic.tar.gz’ saved [2468744/2468744]



### Upload das Sequências que Serão Alinhadas

Nesta seção, faremos o upload das sequências das proteínas da família das NTPDases para o posterior alinhamento. Neste notebook, assumimos que as sequências estão em formato **fasta**.

O arquivo que usaremos está disponível [aqui](https://github.com/grsart/BiomolComp/blob/main/Pratica5/Seqs0.fasta). Clique no link e abra em nova aba para visualizar as sequências.

**Q1 - Consegue antecipar alguma informação sobre como vai ser o alinhamento? O que pode falar sobre os organismos, sobre os tamanhos de sequência?**

In [8]:
seq_file="Seqs0.fasta"
!wget https://raw.githubusercontent.com/grsart/BiomolComp/refs/heads/main/Pratica5/Seqs0.fasta
!ls
!head Seqs0.fasta

--2025-09-25 22:02:09--  https://raw.githubusercontent.com/grsart/BiomolComp/refs/heads/main/Pratica5/Seqs0.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1694 (1.7K) [text/plain]
Saving to: ‘Seqs0.fasta’

Seqs0.fasta         100%[===================>]   1.65K  --.-KB/s    in 0s      

2025-09-25 22:02:09 (13.5 MB/s) - ‘Seqs0.fasta’ saved [1694/1694]

clustalw2				       sample_data
clustalw-2.1-linux-x86_64-libcppstatic	       Seqs0.fasta
clustalw-2.1-linux-x86_64-libcppstatic.tar.gz  Seqs0.fasta.1
>Antibody_Homosapiens-pembro
QVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPNSGGTNYAQKFQGRVTMTRDTSISTAYMELSRLRSDDTAVYYCAKISGSYSFDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSC
>sp|P

###Inspecionando as sequências que serão alinhadas

Nesta próxima seção, vamos inspecionar o nosso arquivo fasta, que contem múltiplas sequências. Vamos usar as funções do Biopython (SeqIO) para isto.

Cada sequência é escaneada pela função *SeqIO.parse*. Delas, podemos listar a própria sequência, o cabeçalho (Id) e o número e resíduos na sequência, por exemplo.

In [9]:
count=0
for seq_record in SeqIO.parse(seq_file, "fasta"):
    count = count+1
    print("SeqID: %s" % seq_record.id)
    print(repr(seq_record.seq))
    print("Sequence Length: %d" % len(seq_record))
print("")
print("Number of sequences found: %d" % count);

SeqID: Antibody_Homosapiens-pembro
Seq('QVQLVQSGAEVKKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPN...KSC')
Sequence Length: 221
SeqID: sp|P43235|CATK_HUMAN
Seq('MWGLKVLLLPVVSFALYPEEILDTHWELWKKTHRKQYNNKVDEISRRLIWEKNL...PKM')
Sequence Length: 329
SeqID: sp|P68871|HBB_HUMAN
Seq('MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDA...KYH')
Sequence Length: 147
SeqID: sp|P50995|ANX11_HUMAN
Seq('MSYPGYPPPPGGYPPAAPGGGPWGGAAYPPPPSMPPIGLDNVATYAGQFNQDYL...GND')
Sequence Length: 505
SeqID: sp|P04406|G3P_HUMAN
Seq('MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHG...SKE')
Sequence Length: 335

Number of sequences found: 5


###Alinhamento das Sequências com o ClustalW

Nesta seção, faremos o alinhamento pelo método progressivo, usando o software *ClustalW*.

No método progressivo, as sequencias são inicialmente alinhadas par a par e uma **distância** entre as sequências é calculada baseada no score do alinhamento (maior score significa maior similaridade e menor distância entre as sequências). Em seguida, um alinhamento múltiplo é construído pela adição progressiva das sequências, iniciando pelas mais próximas.

O programa [ClustalW](http://www.clustal.org/clustal2/) foi descontinuado, mas ainda pode ser obtido. Em seu lugar, foi lançado o programa [Clustal Omega](http://www.clustal.org/omega/), que emprega modelos de Marvok (HMM) para o alinhamento mais rápido e preciso, partcularmente quando o número de sequências a ser alinhado é grande.

Na seção abaixo, o biopython invoca o clustalw2 (que já baixamos e deixamos na pasta local) e passa o arquivo fasta com as múltiplas sequências como argumento.


In [10]:
clustalw_exe = r"./clustalw2"
clustalw_cline = ClustalwCommandline(clustalw_exe, infile=seq_file)
assert os.path.isfile(clustalw_exe), "Clustal W executable missing"
stdout, stderr = clustalw_cline()



### Inspecionando o Alinhamento

Com sorte, o alinhamento já foi realizado pelo ClustalW. Dois arquivos devem ter sido escritos como arquivos de saída do clustalw: um arquivo com estensão .aln e um arquivo com extensão .dnd.

O primeiro arquivo contém o MSA. O segundo contém as distâncias entre as sequências do MSA. Vamos analisar os dois.

Primeiramente, vamos usar o *parser* do BioPython para abrir o arquivo .aln e printar o alinhamento.

In [12]:
from Bio import AlignIO
align = AlignIO.read("Seqs0.aln", "clustal")
for record in align:
  print("%s - %s" % (record.seq, record.id))

---------------------------------QVQLVQSG------------AEVKKPGASVKVSCKASGYTFTGY-YMHWVRQAPGQGLEWMGWINPNSG------------------------------GTNYAQKFQGR-------------VTMTRDTSISTAYMELSRLRSDDTAVYYCAKISGSYSFDYWG-----QGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFP---------------------------------------EPVTVS------WNSG---------ALTSGVHTFPAVLQSS-----------------------------------GLYSLSSVVTVPSSSLGTQTYICN------------VNHKPSNTKVDKKVEPKSC----------------------------------------------------------------------------------------- - Antibody_Homosapiens-pembro
-----------------------------MGKVKVGVNGFGRIG--------RLVTRAAFNSGKVDIVAINDPFIDLNYMVYMFQYDSTHGKFHGTVKAENGK--------------------------LVINGNPITIFQERDPSKIKWGDAGAEYVVESTGVFTTMEKAGAHLQGGAKRVIISAPSADAPMFVMGVNHEKYDNSLKIISNASCTTNCLAPLAKVIHDNFGIVEGLMTTVHAITA---------------------------------------TQKTVDGPSGKLWRDGRGALQNIIPASTGAAKAVGKVIPEL-----------------------------------NGKLTGMAFRVPTANVSVVDLTCR------------LEKPAKYDDIKKVVKQASEGPLKGILGYTEHQVVSSDFNSDTHSSTFDAGAGIALNDHFVKLISWYD

**### Questão 2**

**A) Como ficou o alinhamento? Ele está confiável?**   
  
**B) Existem regiões conservadas? E regiões mal alinhadas?**   
  
**C) Quais são os aspectos das sequências de entrada que explicam esse comportamento?**  


### Analisando as distâncias entre as sequências

Uma vez que usamos um método progressivo e que as distâncias entre as sequências foram calculadas, podemos analisar, graficamente, estas distâncias através da construção de uma árvore. A*LERTA DE SPOILER*: Este é o princípio da análise filogenética que veremos em aulas seguintes.

Vamos usar o Biopython para analisar o arquivo .dnd pela construção de uma árvore de distâncias.

In [14]:
from Bio import Phylo
tree = Phylo.read("Seqs0.dnd", "newick")
Phylo.draw_ascii(tree)

  ________________________________________ Antibody_Homosapiens-pembro
 |
 |        _______________________________________ sp|P43235|CATK_HUMAN
 |   ____|
_|__|    |________________________________________ sp|P50995|ANX11_HUMAN
 |  |
 |  |__________________________________________ sp|P68871|HBB_HUMAN
 |
 |_____________________________________ sp|P04406|G3P_HUMAN



**### Questão 3**

**É possível extrair alguma informação a partir da árvore de distância de sequência acima?**  